# Vaccine Hesitancy for COVID-19: County and local estimates

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_latimes as lat
import glob
import os

In [3]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Get the data from the CDC portal

In [4]:
# https://data.cdc.gov/Vaccinations/Vaccine-Hesitancy-for-COVID-19-County-and-local-es/q9mh-h2tw

In [5]:
url = "https://data.cdc.gov/api/views/q9mh-h2tw/rows.csv?accessType=DOWNLOAD"

In [6]:
df = pd.read_csv(url)

### Clean up the headers

In [7]:
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace(":", "", regex=False)
    .str.replace("/", "_", regex=False)
    .str.replace(",", "_", regex=False)
)

In [8]:
df.drop(
    ["county_boundary", "state_boundary", "geographical_point"], axis=1, inplace=True
)

In [9]:
df["fips_code"] = df["fips_code"].astype(str).str.zfill(5)

In [10]:
df.head()

fips_code                     county_name    state  estimated_hesitant  \
0     01131          Wilcox County, Alabama  ALABAMA                0.23   
1     01129      Washington County, Alabama  ALABAMA                0.23   
2     01133         Winston County, Alabama  ALABAMA                0.22   
3     01127          Walker County, Alabama  ALABAMA                0.23   
4     02013  Aleutians East Borough, Alaska   ALASKA                0.26   

   estimated_strongly_hesitant  social_vulnerability_index_(svi)  \
0                         0.11                              0.93   
1                         0.11                              0.73   
2                         0.11                              0.70   
3                         0.11                              0.75   
4                         0.12                              0.58   

        svi_category  cvac_level_of_concern_for_vaccination_rollout  \
0  Very High Concern                                           0.94   
1       High Concern                                           0.82   
2       High Concern                                           0.80   
3       High Concern                                           0.68   
4   Moderate Concern                                           0.87   

  cvac_level_of_concern  percent_adults_fully_vaccinated_against_covid-19  \
0     Very High Concern                                             0.228   
1     Very High Concern                                             0.192   
2          High Concern                                             0.085   
3          High Concern                                             0.158   
4     Very High Concern                                             0.195   

   percent_hispanic  percent_non-hispanic_american_indian_alaska_native  \
0            0.0053                                             0.0009    
1            0.0146                                             0.0731    
2            0.0315                                             0.0034    
3            0.0249                                             0.0015    
4            0.0901                                             0.4588    

   percent_non-hispanic_asian  percent_non-hispanic_black  \
0                      0.0003                      0.6938   
1                      0.0025                      0.2354   
2                      0.0016                      0.0073   
3                      0.0049                      0.0617   
4                      0.1968                      0.0322   

   percent_non-hispanic_native_hawaiian_pacific_islander  \
0                                             0.0000       
1                                             0.0000       
2                                             0.0005       
3                                             0.0000       
4                                             0.0100       

   percent_non-hispanic_white state_code  
0                      0.2684         AL  
1                      0.6495         AL  
2                      0.9370         AL  
3                      0.8895         AL  
4                      0.1321         AK

### Merge with election results

In [11]:
df_election = pd.read_json(
    "../elections/data/election_results_2020.json", dtype={"county_fips": str}
)

In [12]:
df_election.head()

state_name county_fips     county_name  votes_gop  votes_dem  total_votes  \
0    Alabama       01001  Autauga County      19838       7503        27770   
1    Alabama       01003  Baldwin County      83544      24578       109679   
2    Alabama       01005  Barbour County       5622       4816        10518   
3    Alabama       01007     Bibb County       7525       1986         9595   
4    Alabama       01009   Blount County      24711       2640        27588   

    diff   per_gop   per_dem  per_point_diff winner  
0  12335  0.714368  0.270184        0.444184  Trump  
1  58966  0.761714  0.224090        0.537623  Trump  
2    806  0.534512  0.457882        0.076631  Trump  
3   5539  0.784263  0.206983        0.577280  Trump  
4  22071  0.895716  0.095694        0.800022  Trump

In [13]:
merge = pd.merge(
    df,
    df_election,
    left_on="fips_code",
    right_on="county_fips",
    how="left",
).drop(["county_name_y"], axis=1)

In [14]:
merge.dtypes

fips_code                                                 object
county_name_x                                             object
state                                                     object
estimated_hesitant                                       float64
estimated_strongly_hesitant                              float64
social_vulnerability_index_(svi)                         float64
svi_category                                              object
cvac_level_of_concern_for_vaccination_rollout            float64
cvac_level_of_concern                                     object
percent_adults_fully_vaccinated_against_covid-19         float64
percent_hispanic                                         float64
percent_non-hispanic_american_indian_alaska_native       float64
percent_non-hispanic_asian                               float64
percent_non-hispanic_black                               float64
percent_non-hispanic_native_hawaiian_pacific_islander    float64
percent_non-hispanic_whit

In [15]:
merge.rename(columns={"county_name_x": "county_name"}, inplace=True)

In [16]:
merge[["name", "state_name"]] = merge["county_name"].str.split(", ", expand=True)

In [17]:
merge["estimated_hesitant_pct"] = round(merge["estimated_hesitant"] * 100, 2)

In [18]:
merge.head()

fips_code                     county_name    state  estimated_hesitant  \
0     01131          Wilcox County, Alabama  ALABAMA                0.23   
1     01129      Washington County, Alabama  ALABAMA                0.23   
2     01133         Winston County, Alabama  ALABAMA                0.22   
3     01127          Walker County, Alabama  ALABAMA                0.23   
4     02013  Aleutians East Borough, Alaska   ALASKA                0.26   

   estimated_strongly_hesitant  social_vulnerability_index_(svi)  \
0                         0.11                              0.93   
1                         0.11                              0.73   
2                         0.11                              0.70   
3                         0.11                              0.75   
4                         0.12                              0.58   

        svi_category  cvac_level_of_concern_for_vaccination_rollout  \
0  Very High Concern                                           0.94   
1       High Concern                                           0.82   
2       High Concern                                           0.80   
3       High Concern                                           0.68   
4   Moderate Concern                                           0.87   

  cvac_level_of_concern  percent_adults_fully_vaccinated_against_covid-19  \
0     Very High Concern                                             0.228   
1     Very High Concern                                             0.192   
2          High Concern                                             0.085   
3          High Concern                                             0.158   
4     Very High Concern                                             0.195   

   percent_hispanic  percent_non-hispanic_american_indian_alaska_native  \
0            0.0053                                             0.0009    
1            0.0146                                             0.0731    
2            0.0315                                             0.0034    
3            0.0249                                             0.0015    
4            0.0901                                             0.4588    

   percent_non-hispanic_asian  percent_non-hispanic_black  \
0                      0.0003                      0.6938   
1                      0.0025                      0.2354   
2                      0.0016                      0.0073   
3                      0.0049                      0.0617   
4                      0.1968                      0.0322   

   percent_non-hispanic_native_hawaiian_pacific_islander  \
0                                             0.0000       
1                                             0.0000       
2                                             0.0005       
3                                             0.0000       
4                                             0.0100       

   percent_non-hispanic_white state_code state_name county_fips  votes_gop  \
0                      0.2684         AL    Alabama       01131     1833.0   
1                      0.6495         AL    Alabama       01129     6564.0   
2                      0.9370         AL    Alabama       01133    10195.0   
3                      0.8895         AL    Alabama       01127    26002.0   
4                      0.1321         AK     Alaska         NaN        NaN   

   votes_dem  total_votes     diff   per_gop   per_dem  per_point_diff winner  \
0     4048.0       5903.0  -2215.0  0.310520  0.685753        0.375233  Biden   
1     2258.0       8876.0   4306.0  0.739522  0.254394        0.485128  Trump   
2      974.0      11284.0   9221.0  0.903492  0.086317        0.817175  Trump   
3     4834.0      31170.0  21168.0  0.834200  0.155085        0.679115  Trump   
4        NaN          NaN      NaN       NaN       NaN             NaN    NaN   

                     name  estimated_hesitant_pct  
0           Wilcox County                    23.0  
1      

In [19]:
merge_corr = merge[
    ["fips_code", "county_name", "estimated_hesitant", "per_gop", "winner"]
]

In [20]:
merge_corr[merge_corr.winner == "Trump"].corr(method="pearson")

estimated_hesitant   per_gop
estimated_hesitant            1.000000  0.252561
per_gop                       0.252561  1.000000

In [21]:
alt.Chart(merge).mark_circle(size=40).encode(
    x=alt.X("per_gop", title="% Trump", axis=alt.Axis(tickCount=4, format=("%"))),
    y=alt.Y(
        "estimated_hesitant",
        title="% hesitant",
        axis=alt.Axis(tickCount=4, format=("%")),
    ),
    color=alt.Color(
        "winner", scale=alt.Scale(domain=["Trump", "Biden"], range=["red", "blue"])
    ),
).properties(
    width=800, height=500, title="Relationship between Trump vote and vaccine hesitancy"
)

alt.Chart(...)

### Just California

In [22]:
ca_merge = merge[merge["state"] == "CALIFORNIA"].sort_values(
    "estimated_strongly_hesitant", ascending=False
)

In [23]:
ca_merge["county_name"] = ca_merge["county_name"].str.replace(
    ", California", "", regex=False
)

In [24]:
alt.Chart(ca_merge).mark_circle(size=50).encode(
    x=alt.X("per_gop", title="% Trump", axis=alt.Axis(tickCount=6, format=("%"))),
    y=alt.Y(
        "estimated_hesitant",
        title="% hesitant",
        axis=alt.Axis(tickCount=4, format=("%")),
    ),
    color=alt.Color(
        "winner", scale=alt.Scale(domain=["Trump", "Biden"], range=["red", "blue"])
    ),
    size=alt.Size("total_votes"),
).properties(
    width=800, height=500, title="Relationship between Trump vote and vaccine hesitancy"
)

alt.Chart(...)

### Least hesitant in CA

In [25]:
ca_merge.sort_values("estimated_hesitant", ascending=True).head()

fips_code           county_name       state  estimated_hesitant  \
546     06075  San Francisco County  CALIFORNIA                0.07   
534     06041          Marin County  CALIFORNIA                0.08   
566     06081      San Mateo County  CALIFORNIA                0.08   
647     06085    Santa Clara County  CALIFORNIA                0.08   
623     06013   Contra Costa County  CALIFORNIA                0.09   

     estimated_strongly_hesitant  social_vulnerability_index_(svi)  \
546                         0.03                              0.39   
534                         0.03                              0.27   
566                         0.03                              0.26   
647                         0.03                              0.36   
623                         0.04                              0.41   

         svi_category  cvac_level_of_concern_for_vaccination_rollout  \
546       Low Concern                                           0.52   
534       Low Concern                                           0.31   
566       Low Concern                                           0.40   
647       Low Concern                                           0.42   
623  Moderate Concern                                           0.47   

    cvac_level_of_concern  percent_adults_fully_vaccinated_against_covid-19  \
546      Moderate Concern                                             0.237   
534           Low Concern                                             0.333   
566      Moderate Concern                                             0.264   
647      Moderate Concern                                             0.216   
623      Moderate Concern                                             0.261   

     percent_hispanic  percent_non-hispanic_american_indian_alaska_native  \
546            0.1524                                             0.0019    
534            0.1598                                             0.0017    
566            0.2440                                             0.0015    
647            0.2547                                             0.0017    
623            0.2559                                             0.0020    

     percent_non-hispanic_asian  percent_non-hispanic_black  \
546                      0.3407                      0.0500   
534                      0.0581                      0.0213   
566                      0.2831                      0.0218   
647                      0.3628                      0.0235   
623                      0.1648                      0.0840   

     percent_non-hispanic_native_hawaiian_pacific_islander  \
546                                             0.0034       
534                                             0.0010       
566                                             0.0132       
647                                             0.0032       
623                                             0.0046       

     percent_non-hispanic_white state_code  state_name county_fips  votes_gop  \
546                      0.4051         CA  California       06075    56417.0   
534                      0.7115         CA  California       06041    24612.0   
566                      0.3924         CA  California       06081    75563.0   
647                      0.3154         CA  California       06085   214612.0   
623                      0.4383         CA  California       06013   152877.0   

     votes_dem  total_votes      diff   per_gop   per_dem  per_point_diff  \
546   378156.0     443458.0 -321739.0  0.127221  0.852744        0.725523   
534   128288.0     155801.0 -103676.0  0.157971  0.823409        0.665439   
566   291410.0     374138.0 -215847.0  0.201966  0.778884        0.576918   
647   617967.0     850522.0 -403355.0  0.252330  0.726574        0.474244   
623   416386.0     581230.0 -263509.0  0.263023  0.716388        0.453364   

    winner                  name  estimated_hesitant_pct  
546  Biden  San Francisc

### Most hesitant in CA?

In [26]:
ca_merge.sort_values("estimated_hesitant", ascending=True).tail()

fips_code      county_name       state  estimated_hesitant  \
652     06063    Plumas County  CALIFORNIA                0.15   
540     06049     Modoc County  CALIFORNIA                0.15   
657     06093  Siskiyou County  CALIFORNIA                0.15   
588     06035    Lassen County  CALIFORNIA                0.15   
527     06031     Kings County  CALIFORNIA                0.16   

     estimated_strongly_hesitant  social_vulnerability_index_(svi)  \
652                         0.06                              0.27   
540                         0.06                              0.75   
657                         0.06                              0.72   
588                         0.06                              0.60   
527                         0.06                              0.97   

          svi_category  cvac_level_of_concern_for_vaccination_rollout  \
652        Low Concern                                           0.54   
540       High Concern                                           0.78   
657       High Concern                                           0.68   
588   Moderate Concern                                           0.70   
527  Very High Concern                                           0.75   

    cvac_level_of_concern  percent_adults_fully_vaccinated_against_covid-19  \
652      Moderate Concern                                               NaN   
540          High Concern                                               NaN   
657          High Concern                                             0.215   
588          High Concern                                             0.131   
527          High Concern                                             0.108   

     percent_hispanic  percent_non-hispanic_american_indian_alaska_native  \
652            0.0895                                             0.0144    
540            0.1449                                             0.0418    
657            0.1258                                             0.0320    
588            0.1896                                             0.0274    
527            0.5449                                             0.0085    

     percent_non-hispanic_asian  percent_non-hispanic_black  \
652                      0.0114                      0.0076   
540                      0.0111                      0.0153   
657                      0.0160                      0.0157   
588                      0.0139                      0.0848   
527                      0.0369                      0.0583   

     percent_non-hispanic_native_hawaiian_pacific_islander  \
652                                             0.0046       
540                                             0.0010       
657                                             0.0029       
588                                             0.0080       
527                                             0.0016       

     percent_non-hispanic_white state_code  state_name county_fips  votes_gop  \
652                      0.8314         CA  California       06063     6445.0   
540                      0.7769         CA  California       06049     3109.0   
657                      0.7610         CA  California       06093    13290.0   
588                      0.6543         CA  California       06035     8970.0   
527                      0.3221         CA  California       06031    24072.0   

     votes_dem  total_votes    diff   per_gop   per_dem  per_point_diff  \
652     4561.0      11256.0  1884.0  0.572584  0.405206        0.167377   
540     1150.0       4338.0  1959.0  0.716690  0.265099        0.451591   
657     9593.0      23450.0  3697.0  0.566738  0.409083        0.157655   
588     2799.0      11985.0  6171.0  0.748436  0.233542        0.514894   
527    18699.0      43858.0  5373.0  0.548862  0.426353        0.122509   

    winner             name  estimated_hesitant_pct  
652  Trump    Plumas County                    15.0  
540  Trump 

In [27]:
ca_merge_datawrapper = ca_merge.copy()

In [28]:
ca_merge_datawrapper["estimated_hesitant_pct"] = (
    ca_merge_datawrapper["estimated_hesitant"] * 100
)

---

In [29]:
counties = gpd.read_file("input/counties/usa_counties_clean_simplified.json")

In [30]:
counties.columns = (
    counties.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace(":", "", regex=False)
    .str.replace("/", "_", regex=False)
    .str.replace(",", "_", regex=False)
)

In [31]:
counties.head()

statefp countyfp  countyns  geoid       name  shape_area  \
0      15      003  00365281  15003   Honolulu    0.138087   
1      41      003  01155126  41003     Benton    0.198723   
2      41      011  01135848  41011       Coos    0.471548   
3      06      045  00277287  06045  Mendocino    0.935848   
4      41      041  01135856  41041    Lincoln    0.294925   

                                            geometry  
0  POLYGON ((-157.91418 21.63521, -157.98643 21.6...  
1  POLYGON ((-123.14903 44.72022, -123.17499 44.7...  
2  POLYGON ((-123.81155 42.78884, -123.81150 42.7...  
3  POLYGON ((-123.54446 40.00192, -123.59440 40.0...  
4  POLYGON ((-123.72466 45.04443, -123.74415 45.0...

In [32]:
merge_geo = counties.merge(merge, right_on="fips_code", left_on="geoid")

In [33]:
len(merge_geo)

3142

In [34]:
merge_geo.plot()

In [35]:
merge_geo.to_file("output/vaccine_hesitancy_geo.geojson", driver="GeoJSON")

---

### Just the states

In [36]:
states = pd.read_excel(
    "https://aspe.hhs.gov/system/files/pdf/265341/Predicted-Vaccine-Hesitancy-by-State-PUMA-County.xlsx",
    sheet_name="state_hesitancy_estimates",
    dtype={"State FIPS": "str"},
)

In [37]:
states.rename(
    columns={
        "State FIPS": "fips",
        "State Name": "state",
        "% Estimated Hesitant - March 3 - March 15, 2021": "hesitant",
        "% Estimated Strongly Hesitant - March 3 - March 15, 2021": "strongly_hesitant",
    },
    inplace=True,
)

In [38]:
states["hesitant"] = round(states["hesitant"] * 100, 1)
states["strongly_hesitant"] = round(states["strongly_hesitant"] * 100, 1)

In [39]:
states["fips"] = states["fips"].str.zfill(2)

In [40]:
states.head()

fips       state  hesitant  strongly_hesitant
0   01     Alabama      21.5               10.3
1   02      Alaska      20.4                9.0
2   04     Arizona      17.4               10.2
3   05    Arkansas      21.2               12.6
4   06  California      10.8                4.3

---

## Exports 

In [41]:
ca_merge.sort_values("estimated_hesitant", ascending=True).head(7).to_csv(
    "output/hesitancy/seven_least_hesitant.csv", index=False
)

In [42]:
ca_merge.sort_values("estimated_hesitant", ascending=True).tail(7).to_csv(
    "output/hesitancy/seven_most_hesitant.csv", index=False
)

In [43]:
ca_merge.sort_values("estimated_hesitant", ascending=True).to_csv(
    "output/hesitancy/ca_hesitant_all_counties.csv", index=False
)

In [44]:
ca_merge_datawrapper[
    ["fips_code", "county_name", "estimated_hesitant_pct", "winner"]
].sort_values("estimated_hesitant_pct", ascending=True).to_csv(
    "output/hesitancy/ca_hesitant_datawrapper.csv", index=False
)

In [45]:
states.sort_values("hesitant", ascending=True).to_csv(
    "output/hesitancy/states_hesitantcy.csv", index=False
)

In [46]:
merge[["fips_code", "name", "estimated_hesitant_pct", "winner"]].sort_values(
    "estimated_hesitant_pct", ascending=True
).to_csv("output/hesitancy/usa_counties.csv", index=False)